#Step 0: Introduction


O código inteiro do notebook foi replicado, e os ToDo's foram substituídos pelo código adequado. A explicação do significado de Bag of Wors está no Step 2.1. No Step 6 explico a diferença entre sensitividade e especificidade. Naive-Bayes é implementado no Step 5. No Step 7 comparo o Naive-Bayes com outro algoritmo.

#Step 1.1: Understanding our dataset


In [ ]:
import pandas as pd
# Dataset from - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df = pd.read_table('smsspamcollection/SMSSpamCollection', '\t', names=['label', 'sms_message'] )

# Output printing out first 5 columns
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


#Step 1.2: Data preprocessing



In [ ]:
df['label'] = df.label.map({'ham':0, 'spam':1})

print("Number of rows/columns:")
print (df.shape)
df.head()

Number of rows/columns:
(5572, 2)


,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


#Step 2.1: Bag of Words


O "Bag of Words" é um conceito aplicado em problemas em que conjuntos de palavras precisam ser analisados. Consiste basicamente em contar o número de palavras nos textos e armazenar essas frequências em uma matriz.

Essa matriz geralmente é organizada da seguinte forma: na primeira linha ficam todas as palavras dos textos, e  na primeira coluna ficam os diferentes textos. Nas demais células a_ij, coloca-se o número 1 caso a palavra da coluna j esteja no texto i, e o número 0 caso não esteja. Dessa maneira, conseguimos converter os nossos dados para formato numérico, que é tipo de dado normalmente esperado por algoritmos de machine learning.

Uma maneira simples e eficaz de implementar o Bag of Words é usar o count vectorizer do sklearn, o qual nos permite formatar os dados de maneira mais eficiente, como não diferenciar palavras maiúsculas de minúsculas e remover pontuações.

#Step 2.2: Implementing Bag of Words from scratch


In [ ]:
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

lower_case_documents = []
for i in documents:
    lower_case_documents.append(i.lower())
print(lower_case_documents)

['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


A

In [ ]:
sans_punctuation_documents = []
import string

for i in lower_case_documents:
    sans_punctuation_documents.append(i.translate(str.maketrans('', '', string.punctuation)))
    
print(sans_punctuation_documents)

['hello how are you', 'win money win from home', 'call me now', 'hello call hello you tomorrow']


A

In [ ]:
preprocessed_documents = []
for i in sans_punctuation_documents:
    preprocessed_documents.append(i.split(" "))
print(preprocessed_documents)

[['hello', 'how', 'are', 'you'], ['win', 'money', 'win', 'from', 'home'], ['call', 'me', 'now'], ['hello', 'call', 'hello', 'you', 'tomorrow']]


A

In [ ]:
frequency_list = []
import pprint
from collections import Counter

for i in preprocessed_documents:
    frequency_list.append(Counter(i))
    
pprint.pprint(frequency_list)

[Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1}),
 Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}),
 Counter({'call': 1, 'me': 1, 'now': 1}),
 Counter({'hello': 2, 'call': 1, 'you': 1, 'tomorrow': 1})]


#Step 2.3: Implementing Bag of Words in scikit-learn

In [ ]:
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']

A

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

A

In [ ]:
print(count_vector)

CountVectorizer()


A

In [ ]:
count_vector.fit(documents)
count_vector.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

A

In [ ]:
doc_array = count_vector.transform(documents).toarray()
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

A

In [ ]:
frequency_matrix = pd.DataFrame(doc_array, columns=[count_vector.get_feature_names()])
frequency_matrix

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


#Step 3.1: Training and testing sets

In [ ]:
# split into training and testing sets
# USE from sklearn.model_selection import train_test_split to avoid seeing deprecation warning.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], 
                                                    df['label'], 
                                                    random_state=1)

print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5572
Number of rows in the training set: 4179
Number of rows in the test set: 1393


#Step 3.2: Applying Bag of Words processing to our dataset

In [ ]:
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

#Step 4.1: Bayes Theorem implementation from scratch

In [ ]:
# P(D)
p_diabetes = 0.01

# P(~D)
p_no_diabetes = 0.99

# Sensitivity or P(Pos|D)
p_pos_diabetes = 0.9

# Specificity or P(Neg/~D)
p_neg_no_diabetes = 0.9

# P(Pos)
p_pos = (p_diabetes * p_pos_diabetes) + (p_no_diabetes * (1 - p_neg_no_diabetes))
print('The probability of getting a positive test result P(Pos) is: {}',format(p_pos))

The probability of getting a positive test result P(Pos) is: {} 0.10799999999999998


A

In [ ]:
# P(D|Pos)
p_diabetes_pos = (p_diabetes * p_pos_diabetes)/p_pos
print('Probability of an individual having diabetes, given that that individual got a positive test result is:\
',format(p_diabetes_pos)) 

Probability of an individual having diabetes, given that that individual got a positive test result is: 0.08333333333333336


A

In [ ]:
# P(Pos/~D)
p_pos_no_diabetes = 0.1

# P(~D|Pos)
p_no_diabetes_pos = (p_no_diabetes * p_pos_no_diabetes)/p_pos
print('Probability of an individual not having diabetes, given that that individual got a positive test result is: {}',format(p_no_diabetes_pos))

Probability of an individual not having diabetes, given that that individual got a positive test result is: {} 0.9166666666666669


# Step 4.2: Naive Bayes implementation from scratch

In [ ]:
# P(J)
p_j = 0.5

# P(F/J)
p_j_f = 0.1

# P(I/J)
p_j_i = 0.1

p_j_text = p_j * p_j_f * p_j_i
print(p_j_text)

0.005000000000000001


A

In [ ]:
# P(G)
p_g = 0.5

# P(F/G)
p_g_f = 0.7

# P(I/G)
p_g_i = 0.2

p_g_text = p_g * p_g_f * p_g_i
print(p_g_text)

0.06999999999999999


A

In [ ]:
p_f_i = p_g_text + p_j_text
print('Probability of words freedom and immigration being said are: ', format(p_f_i))

Probability of words freedom and immigration being said are:  0.075


A

In [ ]:
p_j_fi = p_j_text/p_f_i
print('The probability of Jill Stein saying the words Freedom and Immigration: ', format(p_j_fi))

The probability of Jill Stein saying the words Freedom and Immigration:  0.06666666666666668


A

In [ ]:
p_g_fi = p_g_text/p_f_i
print('The probability of Gary Johnson saying the words Freedom and Immigration: ', format(p_g_fi))

The probability of Gary Johnson saying the words Freedom and Immigration:  0.9333333333333332


#Step 5: Naive Bayes implementation using scikit-learn

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB()

A

In [ ]:
predictions = naive_bayes.predict(testing_data)

#Step 6: Evaluating our model

A sensibilidade é basicamente a taxa de verdadeiros positivos, ou seja, a razão entre o número de testes verdadeiros positivos e o número de doentes (que seria a soma entre os verdadeiros positivos e os falsos negativos). Ele reflete a probabilidade do teste ser positivo dado que você possua a doença.

Já a especificidade é a taxa de verdadeiros negativos, isto é, a razão entre o número de testes verdadeiros negativos e o número de pessoas sem a doença (a soma entre os verdadeiras negativos e os falsos positivos).  Indica a probabilidade do teste ser negativo dado que você não possua a doença.

Essas métricas podem ser usadas em testes como os de doenças, ou até mesmo em modelos como o nosso, onde temos uma classificação binária spam/ham. Entretanto, precisamos ficar atentos à tais métricas, pois em alguns cenários seu resultado possa estar adulterado.

Suponha que em uma população, 1000 pessoas foram testadas para saber se estavam portando o corona vírus. Dessas 1000 pessoas, suponha que 50 possuam o vírus, mas por algum motivo o teste é extremamente falho e SEMPRE acusa a presença do vírus. Desse modo, o número de verdadeiros positivos é 50 e o número de falsos negativos é zero, então a sensibilidade é de 100%, mas é claro que o teste é bastante ineficiente.

Por outro lado, se nessa mesma população com as mesmas características o teste SEMPRE acusasse a ausência do vírus, teríamos uma quantidade de 950 verdadeiros negativos e zero falsos positivos, o que indica uma especificidade de 100%, mas novamente fica claro que o teste em questão possuí problema.

Por isso, é sempre importante analisar o contexto do problema para entender quando essas métricas podem ser aplicadas de maneira eficiente.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9885139985642498
Precision score:  0.9720670391061452
Recall score:  0.9405405405405406
F1 score:  0.9560439560439562


#Step 7: Conclusion

Agora, irei comparar o algoritmo usado (Naive-Bayes) com um outro algoritmo de Machine Learning, chamado Decision Tree ou árvore de decisões.


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
dtc.fit(training_data, y_train)

predictions = dtc.predict(testing_data)

print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9641062455132807
Precision score:  0.8534031413612565
Recall score:  0.8810810810810811
F1 score:  0.8670212765957447


Você pode observar que todas as métricas aplicadas ao modelo da árvore de decisões deram resultados inferiores as métricas do modelo de Naive-Bayes. 

Um dos contras do algoritmo de árvores de decisão é se sobreajustar a ruídos, o que pode ser uma das causas da menor avaliação desse algoritmo. Palavras mais comuns, que são usadas em ambos textos de spam ou ham, podem ter prejudicado o desempenho desse algoritmo. Outros contras do algoritmo de árvore de decisão que podem ter contribuído para a menor performance são a separação dos dados através de lineares ortogonais e propensão a overfiting.

Um dos pontos positivos do algoritmo de Naive-Bayes são sua eficiência em datasets pequenos quando as features são independentes, o que parece ser o caso, uma vez que algumas palavras em específico individualmente já são marcadores fortíssimos de SPAM. Portanto, em razão disso, estaria explicado o resultado obtido.